# Creación Variables del Pasivo

In [1]:
# cuentas_out['prom_dias_desde_ult_trans'] = gp_cuentas['dias_desde_ultima_trans'].mean()
# cuentas_out['prom_antiguedad_cuenta'] = gp_cuentas['antiguedad'].mean()

In [2]:
import pandas as pd 
import numpy as np 
import os

pd.set_option('max_columns',None)
#pd.set_option('max_rows',100)

from datetime import datetime, timedelta 
from dateutil import relativedelta

#pd.set_option('max_columns',False)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# path_in="s3://adl-refined-dev-popular/propension/raw_piloto/producto/pasivo/2018/"
# #path_in = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"
# path_out = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"


# for i in range(1,4):
    
#     if i<=9: var_aux = '0'+str(i)
#     else: var_aux = str(i)
        
#     data = pd.read_csv(path_in+var_aux+".csv",sep=",",encoding='utf_8',
#                         dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
    
#     print('Periodo 2020',var_aux)
#     print(data.shape)
    
#     data.to_csv(path_out+var_aux+"2.csv",sep="|",encoding='utf_8', index= False)
    
#     del data

In [4]:
# def func_dates(tabla_final,date):
#     if date == '201907':
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], dayfirst= True)
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], dayfirst= True)
#     elif date in dates2:
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%d/%m/%Y')
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%d/%m/%Y')
#     else:
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y%m%d',errors='coerce')
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y%m%d',errors='coerce')
        
#     return tabla_final

## Creando pasivos Total

In [5]:
# path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"

# str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
#             '202001','202002','202003','202003']

# # str_dates = ['202004']

# product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}


# dates1 = ['201904','201908','201909','201910','201911','201912','202001','202002','202003','202004','202005','202006']
# dates2 = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#          '201901','201902','201903','201905','201906','201907']

# for ix, date in enumerate(str_dates):
    
#     print('Periodo:',date)
#     if ix == 0:
#         tabla_final = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
#                     dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
#         tabla_final.columns = tabla_final.columns.str.lower()
        
#         ## Cambiar tipo producto:
#         tabla_final['periodo'] = int(date)
#         tabla_final['tipo_producto'] = tabla_final['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
#         tabla_final = func_dates(tabla_final,date)
        
#         print(tabla_final.shape)  
        
#     else:
        
#         tabla_temp = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
#                     dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
#         tabla_temp.columns = tabla_temp.columns.str.lower()
        
#         ## Cambiar tipo producto:
#         tabla_temp['periodo'] = int(date)
#         tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
#         tabla_temp = func_dates(tabla_temp,date)
        
#         print(tabla_temp.shape)
        
#         tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)
        
        
#         del tabla_temp

In [6]:
# tabla_final2 = tabla_final.copy() 

In [7]:
import s3fs
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

In [8]:
# -*- coding: utf-8 -*-
PASIVOS_MONEY_COLS = ['saldo',
                      'promedio1',
                      'promedio2',
                      'promedio3',
                      'promedio4',
                      'promedio5',
                      'promedio6']

dict_cc={0:'Nueva',
         1:'Activa',
         2:'Bloqueada',
         3:'Cancelada',
         4:'Inactiva',
         5:'Por Cancelar',
         6:'Saldada',
         7:'Desbloqueada',
         8:'Por Saldar',
         9:'Por Abrir'}

dict_cdt={1:'Captado',
          2:'Depositado',
          3:'Exigible',
          4:'Cobrado',
          5:'Bloquedo',
          6:'Embargado',
          7:'Retenido',
          8:'Anulado',
          9:'Cancelado',
          10:'Canc. Fraccionamiento',
          11:'Canc. Fusion'}

dict_ca={0:'Nueva',
         1:'Activa',
         2:'Bloqueada',
         3:'Cancelada',
         4:'Inactiva',
         5:'Por Cancelar',
         6:'Saldada',
         7:'Desbloqueada',
         8:'Por Saldar',
         9:'Por Abrir'}

def creando_pasivos_antg(input_paths):
    
    periodo = input_paths[0][-6:]
    ## 1. CC
    print('Leyendo CC')
    cc=spark_read_parquet(input_paths[0])
    cc=cc.rename({'acmcc_scotipide':'tipo_id','id_cli':'identificacion','acmcc_ccu_produ':'cod_producto','acmcc_cfeulttra':'fecha_ultima_transaccion','acmcc_cfeapecue':'fecha_apertura','acmcc_csatotman':'saldo','acmcc_promedmen01':'promedio1','acmcc_promedmen02':'promedio2','acmcc_promedmen03':'promedio3','acmcc_promedmen04':'promedio4','acmcc_promedmen05':'promedio5','acmcc_promedmen06':'promedio6','periodo':'periodo'},axis=1)
    cc['nro_cuenta']=cc['cod_producto'].astype('str')+cc['acmcc_ccu_ofici'].astype('str').str.zfill(3)+cc['acmcc_ccunumcue'].astype('str').str.zfill(5)+cc['acmcc_ccudigver'].astype('str') 
    cc["fecha_ultima_transaccion"] = pd.to_datetime(cc["fecha_ultima_transaccion"], format="%Y%m%d", errors="coerce")
    cc["fecha_apertura"] = pd.to_datetime(cc["fecha_apertura"], format="%Y%m%d", errors="coerce")
    
    if cc['acmcc_ccoestcue'].dtypes == 'object':
        cc['acmcc_ccoestcue'] = cc['acmcc_ccoestcue'].astype('int64')
    cc['estado']=[dict_cc.get(i) for i in cc['acmcc_ccoestcue'] ]
    cc=cc[(cc['cod_producto']==110)&(cc['estado'].isin(['Nueva','Activa']))].reset_index().drop(['index'],axis=1)
    cc=cc[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]
    print(cc.dtypes)
    ## 2. CDT
    print('Leyendo CDT')
    cdt=spark_read_parquet(input_paths[1])
    cdt=cdt.rename({'afmde_scotipide':'tipo_id','id_cli':'identificacion','afmde_fcu_produ':'cod_producto','afmde_ffeultmod':'fecha_ultima_transaccion','afmde_ffe_depos':'fecha_apertura','afmde_fva_depos':'saldo','afmde_promedio':'promedio1','periodo':'periodo'},axis=1)
    cdt['nro_cuenta']=cdt['cod_producto'].astype('str')+cdt['afmde_fcu_ofici'].astype('str').str.zfill(3)+cdt['afmde_fcunumcue'].astype('str').str.zfill(7)+cdt['afmde_fcudigver'].astype('str') 
    cdt["fecha_ultima_transaccion"] = pd.to_datetime(cdt["fecha_ultima_transaccion"], format="%Y%m%d", errors="coerce")
    cdt["fecha_apertura"] = pd.to_datetime(cdt["fecha_apertura"], format="%Y%m%d", errors="coerce")
    
    if cdt['afmde_fco_estde'].dtypes == 'object':
        cdt['afmde_fco_estde'] = cdt['afmde_fco_estde'].astype('int64')
    cdt['estado']=[dict_cdt.get(i) for i in cdt['afmde_fco_estde']]
    
    cdt['promedio2']=np.nan
    cdt['promedio3']=np.nan
    cdt['promedio4']=np.nan
    cdt['promedio5']=np.nan
    cdt['promedio6']=np.nan

    cdt=cdt[(cdt['estado'].isin(['Captado','Depositado','Exigible']))].reset_index().drop(['index'],axis=1)
    cdt=cdt[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]
    print(cdt.dtypes)
    ## 3. CA
    print('Leyendo CA')
    ca=spark_read_parquet(input_paths[2])
    ca=ca.rename({'ahmah_scotipide':'tipo_id','id_cli':'identificacion','ahmah_hcu_produ':'cod_producto','ahmah_hfeulttra':'fecha_ultima_transaccion','ahmah_hfeapecue':'fecha_apertura','ahmah_hsatotman':'saldo','ahmah_hvepromen01':'promedio1','ahmah_hvepromen02':'promedio2','ahmah_hvepromen03':'promedio3','ahmah_hvepromen04':'promedio4','ahmah_hvepromen05':'promedio5','ahmah_hvepromen06':'promedio6','periodo':'periodo'},axis=1)
    ca['nro_cuenta']=ca['cod_producto'].astype('str')+ca['ahmah_hcu_ofici'].astype('str').str.zfill(3)+ca['ahmah_hcunumcue'].astype('str').str.zfill(5)+ca['ahmah_hcudigver'].astype('str')  
    ca["fecha_ultima_transaccion"] = pd.to_datetime(ca["fecha_ultima_transaccion"], format="%Y-%m-%d", errors="coerce")
    ca["fecha_apertura"] = pd.to_datetime(ca["fecha_apertura"], format="%Y-%m-%d", errors="coerce")
    
    if ca['ahmah_hcoestcue'].dtypes == 'object':
        ca['ahmah_hcoestcue'] = ca['ahmah_hcoestcue'].astype('int64')
    ca['estado']=[dict_ca.get(i) for i in ca['ahmah_hcoestcue']]

    ca=ca[(ca['estado'].isin(['Nueva','Activa']))].reset_index().drop(['index'],axis=1)
    ca=ca[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]
    print(ca.dtypes)
    ## 4. Apppend
    print('Append archivos')
    total=cc.append(cdt).append(ca).reset_index().drop(['index'],axis=1)
    total['fecha'] = periodo
    total['fecha'] = pd.to_datetime(total['fecha'], format='%Y%m')
    total['cod_producto']=total['cod_producto'].astype('str')

    total['tipo_ident_hom']=np.where(total['tipo_id']==1,'1000003',
                                       np.where(total['tipo_id']==2,'1000005',
                                               np.where(total['tipo_id']==3,'1000007',
                                                       np.where(total['tipo_id']==4,'1000002',
                                                               np.where(total['tipo_id']==5,'1000004',
                                                                         np.where(total['tipo_id']==6,'1000006',
                                                                                   np.where(total['tipo_id']==7,'1000007',
                                                                                       np.where(total['tipo_id']==8,'1000008',
                                                                                                np.where(total['tipo_id']==9,'1000009','99999')))))))))

    frame_total=total
    
    # Cast
    if frame_total['identificacion'].isnull().sum() > 0:
        frame_total = frame_total.loc[frame_total['id_cli'].notnull()]

    frame_total['identificacion'] = frame_total['identificacion'].astype(np.int64)
    frame_total['tipo_ident_hom'] = frame_total['tipo_ident_hom'].astype(np.int64)
    
    return frame_total

In [9]:
def create_dates_v2(fecha, n, day = '01'):
    """
    Descripcion: Función para generar n fechas hacia el adelante o atras en meses
    Parámetros: fecha (string) en formato yyyymm, corresponde a la fecha base
                n (int), corresponde a la cantidad de meses que se quiere devolver
                day (string), día del mes en que inicia el proceso, se recomienda no cambiarlo
    Retorno: lista de fechas (string)
    ejemplo: create_dates_v2('202006', -2)
    """
    n =int(n)
    fechas = []
    fecha_hoy = str(fecha) + day
    start = datetime.strptime(fecha_hoy, '%Y%m%d')
    temp = start
    if n <= 0:
        x = n *-1
        fechas.append(fecha)
        for ind in range(x):
            end = temp - timedelta(days = temp.day)
            fecha_final = end.strftime('%Y%m%d')[:6]
            fechas.append(fecha_final)
            temp = end
    if n > 0:
        x = n
        for ind in range(x + 1):
            end = temp + timedelta(days = 35)
            end = end - timedelta(days = end.day)
            fecha_final = end.strftime('%Y%m%d')[:6]
            fechas.append(fecha_final)
            temp = end
    return sorted(fechas, reverse=False)

In [10]:
end_period = '202110'
create_dates_v2(end_period, -2)

['202108', '202109', '202110']

In [11]:
str_dates = sorted(create_dates_v2(end_period, -2), reverse=False)
print(str_dates)

['202108', '202109', '202110']


In [12]:
#path_in="s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo/"

# str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
#             '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012']

product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}


for ix, date in enumerate(str_dates):
    
    pasivos_path = [os.path.join('s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cc/productos_pasivo-sfb-cc_aaatr_M'+date),
                os.path.join('s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cdts/productos_pasivo-sfb-cdts_aaatr_M'+date),
                os.path.join('s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-ca/productos_pasivo-sfb-ca_aaatr_M'+date)]

    
    #tabla_temp = read_parquet(date)
    tabla_temp = creando_pasivos_antg(pasivos_path)
    tabla_temp.columns = tabla_temp.columns.str.lower()

    ## Cambiar tipo producto:
    tabla_temp['periodo'] = int(date)
    tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)

    print(tabla_temp.shape)
    
    print('Periodo:',date)
    if ix == 0:
        tabla_final = tabla_temp
    else:
        tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)
         
    del tabla_temp

Leyendo CC
tipo_id                              int64
identificacion                      object
cod_producto                         int64
nro_cuenta                          object
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                              float64
promedio1                          float64
promedio2                          float64
promedio3                          float64
promedio4                          float64
promedio5                          float64
promedio6                          float64
periodo                              int64
dtype: object
Leyendo CDT
tipo_id                             object
identificacion                      object
cod_producto                        object
nro_cuenta                          object
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo            

In [13]:
tabla_final

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
0,6,494431585,110,110251114021,2021-08-30,1990-06-01,Activa,38200006.04,6429683.42,8723625.35,13118193.8,18045178.39,4617878.1,27252841.39,202108,2021-08-01,1000006,corriente
1,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
2,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
3,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
4,4,800070728,110,110311123244,2021-08-26,1989-09-01,Activa,537497.14,28340626.03,15995450.23,37076081.17,64444854.0,35416349.53,45071462.87,202108,2021-08-01,1000002,corriente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987022,5,95918,210,210039013362,2021-11-02,2000-06-16,Activa,5768788.38,1604078,1352693,1239335,6223973,1369123,1595483,202110,2021-10-01,99999,ahorro
1987023,5,97158,230,230460853138,2021-11-02,2004-09-14,Activa,6676999.18,4335085,4821394,6342009,8687466,5148749,6219764,202110,2021-10-01,99999,ahorro
1987024,5,977082,230,230264002965,2021-11-02,2019-12-03,Activa,802703.89,88441,215246,665889,201250,516654,157072,202110,2021-10-01,99999,ahorro
1987025,5,98224,230,230585109374,2021-11-02,2007-11-30,Activa,7474643.24,3130472,1278988,4646205,5033006,1318005,4126306,202110,2021-10-01,99999,ahorro


In [14]:
## Cast dates
tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format="%Y-%m-%d", errors="coerce")
tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format="%Y-%m-%d", errors="coerce")

In [15]:
# tabla_final2 = tabla_final2.drop_duplicates()
# tabla_final = tabla_final.drop_duplicates()

# tabla_final = pd.concat([tabla_final,tabla_final2], ignore_index=True)

In [16]:
tabla_final.columns

Index(['tipo_id', 'identificacion', 'cod_producto', 'nro_cuenta',
       'fecha_ultima_transaccion', 'fecha_apertura', 'estado', 'saldo',
       'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5',
       'promedio6', 'periodo', 'fecha', 'tipo_ident_hom', 'tipo_producto'],
      dtype='object')

In [17]:
tabla_final.reset_index(inplace=True)

In [18]:
tabla_final = tabla_final.drop(columns={'index'})

In [19]:
print(tabla_final.shape)
tabla_final

(1987027, 18)


,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
0,6,494431585,110,110251114021,2021-08-30,1990-06-01,Activa,38200006.04,6429683.42,8723625.35,13118193.8,18045178.39,4617878.1,27252841.39,202108,2021-08-01,1000006,corriente
1,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
2,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
3,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
4,4,800070728,110,110311123244,2021-08-26,1989-09-01,Activa,537497.14,28340626.03,15995450.23,37076081.17,64444854.0,35416349.53,45071462.87,202108,2021-08-01,1000002,corriente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987022,5,95918,210,210039013362,2021-11-02,2000-06-16,Activa,5768788.38,1604078,1352693,1239335,6223973,1369123,1595483,202110,2021-10-01,99999,ahorro
1987023,5,97158,230,230460853138,2021-11-02,2004-09-14,Activa,6676999.18,4335085,4821394,6342009,8687466,5148749,6219764,202110,2021-10-01,99999,ahorro
1987024,5,977082,230,230264002965,2021-11-02,2019-12-03,Activa,802703.89,88441,215246,665889,201250,516654,157072,202110,2021-10-01,99999,ahorro
1987025,5,98224,230,230585109374,2021-11-02,2007-11-30,Activa,7474643.24,3130472,1278988,4646205,5033006,1318005,4126306,202110,2021-10-01,99999,ahorro


In [20]:
tabla_final.count()

tipo_id                     1987027
identificacion              1987027
cod_producto                1987027
nro_cuenta                  1987027
fecha_ultima_transaccion    1962000
fecha_apertura              1987018
estado                      1987027
saldo                       1987027
promedio1                   1987027
promedio2                   1959238
promedio3                   1959238
promedio4                   1959238
promedio5                   1959238
promedio6                   1959238
periodo                     1987027
fecha                       1987027
tipo_ident_hom              1987027
tipo_producto               1987021
dtype: int64

In [21]:
tabla_final.tail()

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
1987022,5,95918,210,210039013362,2021-11-02,2000-06-16,Activa,5768788.38,1604078,1352693,1239335,6223973,1369123,1595483,202110,2021-10-01,99999,ahorro
1987023,5,97158,230,230460853138,2021-11-02,2004-09-14,Activa,6676999.18,4335085,4821394,6342009,8687466,5148749,6219764,202110,2021-10-01,99999,ahorro
1987024,5,977082,230,230264002965,2021-11-02,2019-12-03,Activa,802703.89,88441,215246,665889,201250,516654,157072,202110,2021-10-01,99999,ahorro
1987025,5,98224,230,230585109374,2021-11-02,2007-11-30,Activa,7474643.24,3130472,1278988,4646205,5033006,1318005,4126306,202110,2021-10-01,99999,ahorro
1987026,5,98799,230,230261107494,2021-10-14,2009-02-12,Activa,1402578.56,669790,1321098,451853,1405849,1591987,377822,202110,2021-10-01,99999,ahorro


In [22]:
tabla_final.dtypes

tipo_id                             object
identificacion                       int64
cod_producto                        object
nro_cuenta                          object
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                               object
promedio1                           object
promedio2                           object
promedio3                           object
promedio4                           object
promedio5                           object
promedio6                           object
periodo                              int64
fecha                       datetime64[ns]
tipo_ident_hom                       int64
tipo_producto                       object
dtype: object

In [23]:
tabla_final.groupby('periodo').size()

periodo
202108    667003
202109    663544
202110    656480
dtype: int64

In [24]:
## eliminando comas:

# tabla_final['saldo'] = [float(str(x).replace(',','.')) for x in tabla_final['saldo']]
# tabla_final['promedio1'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio1']]
# tabla_final['promedio2'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio2']]
# tabla_final['promedio3'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio3']]
# tabla_final['promedio4'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio4']]
# tabla_final['promedio5'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio5']]
# tabla_final['promedio6'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio6']]


In [25]:
tabla_final.head()

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
0,6,494431585,110,110251114021,2021-08-30,1990-06-01,Activa,38200006.04,6429683.42,8723625.35,13118193.8,18045178.39,4617878.1,27252841.39,202108,2021-08-01,1000006,corriente
1,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
2,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
3,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
4,4,800070728,110,110311123244,2021-08-26,1989-09-01,Activa,537497.14,28340626.03,15995450.23,37076081.17,64444854.0,35416349.53,45071462.87,202108,2021-08-01,1000002,corriente


In [26]:
## Cast variables

tabla_final = tabla_final[tabla_final['identificacion'].notnull()]
tabla_final = tabla_final[tabla_final['nro_cuenta'].isna()==False]

tabla_final['identificacion'] = tabla_final['identificacion'].astype('int64')
tabla_final['nro_cuenta'] = tabla_final['nro_cuenta'].astype('int64')
tabla_final['cod_producto'] = tabla_final['cod_producto'].astype('int64')
tabla_final['tipo_id'] = tabla_final['tipo_id'].astype('int64')

# tabla_final['saldo'] = tabla_final['saldo'].astype('float64')
# tabla_final['promedio1'] = tabla_final['promedio1'].astype('float64')
# tabla_final['promedio2'] = tabla_final['promedio2'].astype('float64')
# tabla_final['promedio3'] = tabla_final['promedio3'].astype('float64')
# tabla_final['promedio4'] = tabla_final['promedio4'].astype('float64')
# tabla_final['promedio5'] = tabla_final['promedio5'].astype('float64')
# tabla_final['promedio6'] = tabla_final['promedio6'].astype('float64')
tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y-%m-%d')
tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y-%m-%d')

In [27]:
tabla_final[tabla_final['periodo'] == 202108].count()

tipo_id                     667003
identificacion              667003
cod_producto                667003
nro_cuenta                  667003
fecha_ultima_transaccion    658307
fecha_apertura              667000
estado                      667003
saldo                       667003
promedio1                   667003
promedio2                   657384
promedio3                   657384
promedio4                   657384
promedio5                   657384
promedio6                   657384
periodo                     667003
fecha                       667003
tipo_ident_hom              667003
tipo_producto               667001
dtype: int64

In [28]:
tabla_final.dtypes

tipo_id                              int64
identificacion                       int64
cod_producto                         int64
nro_cuenta                           int64
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                               object
promedio1                           object
promedio2                           object
promedio3                           object
promedio4                           object
promedio5                           object
promedio6                           object
periodo                              int64
fecha                       datetime64[ns]
tipo_ident_hom                       int64
tipo_producto                       object
dtype: object

In [29]:
tabla_final[tabla_final['periodo'] == 202108].head()

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
0,6,494431585,110,110251114021,2021-08-30,1990-06-01,Activa,38200006.04,6429683.42,8723625.35,13118193.8,18045178.39,4617878.1,27252841.39,202108,2021-08-01,1000006,corriente
1,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
2,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
3,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.9,73277506.87,76566591.29,79171445.0,77768772.16,202108,2021-08-01,1000002,corriente
4,4,800070728,110,110311123244,2021-08-26,1989-09-01,Activa,537497.14,28340626.03,15995450.23,37076081.17,64444854.0,35416349.53,45071462.87,202108,2021-08-01,1000002,corriente


In [30]:
tabla_final = tabla_final[tabla_final['identificacion'] != -1]

In [31]:
tabla_final['fecha'] = pd.to_datetime(tabla_final['periodo'], format='%Y%m')

In [32]:
periodo = 202110

In [33]:
money = ['saldo', 'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5', 'promedio6']

for col in money:
    tabla_final[col] = tabla_final[col].astype('float64')

In [34]:
tabla_final.dtypes

tipo_id                              int64
identificacion                       int64
cod_producto                         int64
nro_cuenta                           int64
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                              float64
promedio1                          float64
promedio2                          float64
promedio3                          float64
promedio4                          float64
promedio5                          float64
promedio6                          float64
periodo                              int64
fecha                       datetime64[ns]
tipo_ident_hom                       int64
tipo_producto                       object
dtype: object

In [35]:
path_in="s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_tot_M"

tabla_final.to_parquet(path_in+str(periodo), engine='pyarrow',index=False)

# Cargando Data

In [36]:
import gc
gc.collect()

308

In [37]:
path_in = "s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_tot_M"

In [38]:
pasivo_201803_202003 = pd.read_parquet(path_in+str(periodo), engine='pyarrow')

In [39]:
pasivo_201803_202003.head()

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto
0,6,494431585,110,110251114021,2021-08-30,1990-06-01,Activa,38200006.04,6429683.42,8723625.35,13118193.80,18045178.39,4617878.10,27252841.39,202108,2021-08-01,1000006,corriente
1,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.90,73277506.87,76566591.29,79171445.00,77768772.16,202108,2021-08-01,1000002,corriente
2,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.90,73277506.87,76566591.29,79171445.00,77768772.16,202108,2021-08-01,1000002,corriente
3,4,800074637,110,110311124598,2021-08-30,1990-11-01,Activa,22345211.09,52766267.65,34706273.90,73277506.87,76566591.29,79171445.00,77768772.16,202108,2021-08-01,1000002,corriente
4,4,800070728,110,110311123244,2021-08-26,1989-09-01,Activa,537497.14,28340626.03,15995450.23,37076081.17,64444854.00,35416349.53,45071462.87,202108,2021-08-01,1000002,corriente


In [40]:
## Validación de duplicados a nivel 'id_cliente','nrooblig','fecha'

In [41]:
pasivo_201803_202003=pasivo_201803_202003.sort_values(['identificacion','nro_cuenta','periodo'])

In [42]:
#Se eliminan duplicados
pasivo_201803_202003=pasivo_201803_202003.drop_duplicates(subset=['identificacion','nro_cuenta','periodo'],keep='first')

In [43]:
pasivo_201803_202003.shape

(1927396, 18)

In [44]:
pasivo_201803_202003['fecha_pasivo']=pasivo_201803_202003['periodo'].astype(int)

In [45]:
##Creacion variables SMLV

In [46]:
pasivo_201803_202003['ano_po'] = pasivo_201803_202003['periodo'].astype('str').str[0:4].astype(int)

#Se asigna el salario minimo mensual vigente de 2018 y 2019
pasivo_201803_202003['SMLV'] = np.where(pasivo_201803_202003['ano_po'] == 2018, 781242,
                                        np.where(pasivo_201803_202003['ano_po'] == 2019, 828116,
                                                 np.where(pasivo_201803_202003['ano_po'] == 2020, 877803,
                                                          np.where(pasivo_201803_202003['ano_po'] == 2021, 908526,
                                                          np.nan))))


In [47]:
pasivo_201803_202003['dia_fecha_corte'] = pasivo_201803_202003['fecha']+timedelta(days=37) ##TODO: Confirmar días

In [48]:
pasivo_201803_202003

,tipo_id,identificacion,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,fecha,tipo_ident_hom,tipo_producto,fecha_pasivo,ano_po,SMLV,dia_fecha_corte
660822,1,946,230,230039143292,2021-07-19,2018-02-08,Activa,5212910.66,3097272.0,4475735.0,2343217.0,3590827.0,2630864.0,3546842.0,202108,2021-08-01,1000003,ahorro,202108,2021,908526.0,2021-09-07
1259410,1,946,230,230039143292,2021-09-22,2018-02-08,Activa,2385623.96,3782063.0,3097272.0,4475735.0,2343217.0,3590827.0,2630864.0,202109,2021-09-01,99999,ahorro,202109,2021,908526.0,2021-10-08
1985523,1,946,230,230039143292,2021-09-22,2018-02-08,Activa,4571847.36,2597278.0,3782063.0,3097272.0,4475735.0,2343217.0,3590827.0,202110,2021-10-01,99999,ahorro,202110,2021,908526.0,2021-11-07
21141,1,1131,300,30059000059592,NaT,2021-07-13,Depositado,348596.00,348596.0,NaN,NaN,NaN,NaN,NaN,202108,2021-08-01,99999,cdt,202108,2021,908526.0,2021-09-07
687656,1,1131,300,30059000059592,NaT,2021-07-13,Depositado,348596.00,348596.0,NaN,NaN,NaN,NaN,NaN,202109,2021-09-01,99999,cdt,202109,2021,908526.0,2021-10-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711152,4,99092300729,250,250251104980,2010-02-19,2007-12-17,Activa,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202109,2021-09-01,99999,ahorro,202109,2021,908526.0,2021-10-08
1368290,4,99092300729,250,250251104980,2010-02-19,2007-12-17,Activa,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202110,2021-10-01,99999,ahorro,202110,2021,908526.0,2021-11-07
20297,4,99110900810,300,30016000023269,NaT,2021-02-06,Depositado,8900000.00,8900000.0,NaN,NaN,NaN,NaN,NaN,202108,2021-08-01,99999,cdt,202108,2021,908526.0,2021-09-07
687342,4,99110900810,300,30016000023269,NaT,2021-02-06,Depositado,8900000.00,8900000.0,NaN,NaN,NaN,NaN,NaN,202109,2021-09-01,99999,cdt,202109,2021,908526.0,2021-10-08


### Creación base a nivel id_cliente, fecha

In [49]:
pasivo_201803_202003_cli_fec=pd.DataFrame(pasivo_201803_202003.groupby(['tipo_id','identificacion','periodo'])[['SMLV','fecha_pasivo']].max())

In [50]:
pasivo_201803_202003_cli_fec.shape

(1824650, 2)

In [51]:
pasivo_201803_202003_cli_fec.head()

SMLV  fecha_pasivo
tipo_id identificacion periodo                        
1       946            202108   908526.0        202108
                       202109   908526.0        202109
                       202110   908526.0        202110
        1131           202108   908526.0        202108
                       202109   908526.0        202109

### Variables Pasivo

In [52]:
# Número de cuentas totales vigentes al punto de observación
# Número de cuentas vigentes al punto de observación tipo cta Y
# Dias desde la última transacción
# Antigüedad del cliente con ctas del pasivo
# meses desde la última apertura de cta
# Saldo total a fin de mes
# Saldo maximo a fin de mes
# Saldo mínimo a fin de mes
# Saldo total x meses atrás
# Saldo promedio últimos x meses
# Saldo promedio últimos x meses por tipo cta Y
# tipo de cuenta Digital, nueva, etc.

In [53]:
pasivo_201803_202003.columns

Index(['tipo_id', 'identificacion', 'cod_producto', 'nro_cuenta',
       'fecha_ultima_transaccion', 'fecha_apertura', 'estado', 'saldo',
       'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5',
       'promedio6', 'periodo', 'fecha', 'tipo_ident_hom', 'tipo_producto',
       'fecha_pasivo', 'ano_po', 'SMLV', 'dia_fecha_corte'],
      dtype='object')

In [54]:
## Agrupaciones
gp_pasv = pasivo_201803_202003.groupby(['tipo_id','identificacion','periodo'])
# gp_pasv_cc = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='corriente'].groupby(['tipo_id','identificacion','periodo'])
gp_pasv_ca = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='ahorro'].groupby(['tipo_id','identificacion','periodo'])
# gp_pasv_cdt = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='cdt'].groupby(['tipo_id','identificacion','periodo'])

In [55]:
## Antiguedad Total
pasivo_201803_202003_cli_fec['pasv_antig_total'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].min())/np.timedelta64(30, 'D')
## Numero meses ultima apertura
pasivo_201803_202003_cli_fec['pasv_num_meses_ult_apertura'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].max())/np.timedelta64(30, 'D')
## Dia última transacción
pasivo_201803_202003_cli_fec['pasv_dias_desde_ultima_trans'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_ultima_transaccion'].max())/np.timedelta64(1, 'D')
#

## Saldo Total por cartera fin de mes
pasivo_201803_202003_cli_fec['pasv_saldo_tot_fin_mes_smlv'] = (gp_pasv['saldo'].sum())/(gp_pasv['SMLV'].max())
# pasivo_201803_202003_cli_fec['pasv_saldo_cc_fin_mes_smlv'] = (gp_pasv_cc['saldo'].sum())/(gp_pasv_cc['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].sum())/(gp_pasv_ca['SMLV'].max())
# pasivo_201803_202003_cli_fec['pasv_saldo_cdt_fin_mes_smlv'] = (gp_pasv_cdt['saldo'].sum())/(gp_pasv_cdt['SMLV'].max())

# Creando fecha
pasivo_201803_202003_cli_fec.reset_index(inplace=True)
pasivo_201803_202003_cli_fec['fecha'] = pd.to_datetime(pasivo_201803_202003_cli_fec['periodo'], format='%Y%m')

In [56]:
def gp_func(ids, date):
    fecha = pd.to_datetime(date, format='%Y%m')
    n_meses = [1]
    
    pasv_out = ids.set_index(['tipo_id','identificacion']) # this is different from the actual prod code
    #pasv_out = ids # this is different from the actual prod code
    
    for n in n_meses:
        col_name = 'pasv_saldo_total_'+str(n)+'mes_atras'
#         col_name_cc = 'pasv_saldo_cc_'+str(n)+'mes_atras'
        col_name_ca = 'pasv_saldo_ca_'+str(n)+'mes_atras'
#         col_name_cdt = 'pasv_saldo_cdt_'+str(n)+'mes_atras'
        date_past = fecha - timedelta(days=n*30)

        gp = pasivo_201803_202003_cli_fec[(pasivo_201803_202003_cli_fec['fecha']>date_past)&(pasivo_201803_202003_cli_fec['fecha']<=fecha)].groupby(['tipo_id','identificacion'])
        
        pasv_out[col_name] = gp['pasv_saldo_tot_fin_mes_smlv'].sum()/n
#         pasv_out[col_name_cc] = gp['pasv_saldo_cc_fin_mes_smlv'].sum()/n
        pasv_out[col_name_ca] = gp['pasv_saldo_ca_fin_mes_smlv'].sum()/n
#         pasv_out[col_name_cdt] = gp['pasv_saldo_cdt_fin_mes_smlv'].sum()/n
        
    pasv_out['periodo'] = int(date)
    #pasv_out.index.rename('identificacion', inplace=True)
    pasv_out.reset_index(inplace=True)
    
    return pasv_out

In [57]:
for ix, date in enumerate(str_dates):
    print(date)
    ids = pasivo_201803_202003_cli_fec.loc[pasivo_201803_202003_cli_fec['periodo']==int(date), ['tipo_id','identificacion']].drop_duplicates()
    
    if ix == 0:
        pasv_out = gp_func(ids, date)
    else:    
        pasv_temp = gp_func(ids, date)
        pasv_out = pd.concat([pasv_out, pasv_temp], ignore_index=True)

        
# ## Set_index
# pasivo_201803_202003_cli_fec.set_index(['id_cliente','periodo'], inplace=True)
# pasv_out.set_index(['id_cliente','periodo'], inplace=True)

202108
202109
202110


In [58]:
## Set_index
pasivo_201803_202003_cli_fec.set_index(['tipo_id','identificacion','periodo'], inplace=True)
pasv_out.set_index(['tipo_id','identificacion','periodo'], inplace=True)

In [59]:
pasivo_201803_202003_cli_fec_tot = pasivo_201803_202003_cli_fec.join(pasv_out, how='left')
print(pasv_out.shape)
print(pasivo_201803_202003_cli_fec.shape)
print(pasivo_201803_202003_cli_fec_tot.shape)

pasivo_201803_202003_cli_fec_tot.head()

(1824650, 2)
(1824650, 8)
(1824650, 10)


SMLV  fecha_pasivo  pasv_antig_total  \
tipo_id identificacion periodo                                             
1       946            202108   908526.0        202108         43.566667   
                       202109   908526.0        202109         44.600000   
                       202110   908526.0        202110         45.600000   
        1131           202108   908526.0        202108          1.866667   
                       202109   908526.0        202109          2.900000   

                                pasv_num_meses_ult_apertura  \
tipo_id identificacion periodo                                
1       946            202108                     43.566667   
                       202109                     44.600000   
                       202110                     45.600000   
        1131           202108                      1.866667   
                       202109                      2.900000   

                                pasv_dias_desde_ultima_trans  \
tipo_id identificacion periodo                                 
1       946            202108                           50.0   
                       202109                           16.0   
                       202110                           46.0   
        1131           202108                            NaN   
                       202109                            NaN   

                                pasv_saldo_tot_fin_mes_smlv  \
tipo_id identificacion periodo                                
1       946            202108                      5.737767   
                       202109                      2.625818   
                       202110                      5.032159   
        1131           202108                      0.383694   
                       202109                      0.383694   

                                pasv_saldo_ca_fin_mes_smlv      fecha  \
tipo_id identificacion periodo                                          
1       946            202108                     5.737767 2021-08-01   
                       202109                     2.625818 2021-09-01   
                       202110                     5.032159 2021-10-01   
        1131           202108                          NaN 2021-08-01   
                       202109                          NaN 2021-09-01   

                                pasv_saldo_total_1mes_atras  \
tipo_id identificacion periodo                                
1       946            202108                      5.737767   
                       202109                      2.625818   
                       202110                      5.032159   
        1131           202108                      0.383694   
                       202109                      0.383694   

                                pasv_saldo_ca_1mes_atras  
tipo_id identificacion periodo                            
1       946            202108                   5.737767  
                       202109                   2.625818  
                       202110                   5.032159  
        1131           202108                   0.000000  
                       202109                   0.000000

In [60]:
pasivo_201803_202003_cli_fec_tot.reset_index(inplace=True)

In [61]:
pasivo_201803_202003_cli_fec_tot = pasivo_201803_202003_cli_fec_tot.drop(['SMLV','periodo','fecha','pasv_saldo_ca_fin_mes_smlv','pasv_saldo_tot_fin_mes_smlv'], axis=1)

In [62]:
pasivo_201803_202003_cli_fec_tot.tail(50)

,tipo_id,identificacion,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
1824600,7,1220218332,202108,7.600000,7.600000,12.0,0.005761,0.005761
1824601,7,1220218332,202109,8.633333,8.633333,12.0,0.005618,0.005618
1824602,7,1220218332,202110,9.633333,9.633333,10.0,0.009768,0.009768
1824603,7,1221979188,202108,17.933333,17.933333,61.0,0.657291,0.657291
1824604,7,1221979188,202109,18.966667,18.966667,14.0,0.644528,0.644528
1824605,7,1221979188,202110,19.966667,19.966667,7.0,0.733453,0.733453
1824606,7,1222113284,202108,53.200000,53.200000,25.0,0.013501,0.013501
1824607,7,1222113284,202109,54.233333,54.233333,22.0,0.006556,0.006556
1824608,7,1222113284,202110,55.233333,55.233333,52.0,0.006556,0.006556
1824609,7,1230340220,202108,61.800000,61.800000,48.0,0.716472,0.716472


In [63]:
pasivo_201803_202003_cli_fec_tot.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_M"+str(periodo),engine='pyarrow', index=False)

In [64]:
prueba = pasivo_201803_202003_cli_fec_tot[pasivo_201803_202003_cli_fec_tot['fecha_pasivo'] == periodo]

In [65]:
prueba.head()

,tipo_id,identificacion,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
2,1,946,202110,45.600000,45.600000,46.0,5.032159,5.032159
5,1,1131,202110,0.833333,0.833333,NaN,0.383694,0.000000
8,1,1238,202110,246.966667,246.966667,34.0,0.008672,0.008672
11,1,1272,202110,230.633333,230.633333,128.0,0.000002,0.000002
14,1,1334,202110,204.133333,204.133333,4629.0,0.965219,0.965219


In [66]:
pasivo_201803_202003_cli_fec_tot['fecha_pasivo'].value_counts().sort_index()

202108    612648
202109    609139
202110    602863
Name: fecha_pasivo, dtype: int64